In [ ]:
import numpy as np 
from timeit import default_timer as timer
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from sklearn.metrics import f1_score 
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape.MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler


In [ ]:
with open('train_dataset.pickle', 'rb') as f:data = pickle.load(f, encoding='latin1') 
data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

data['x_train'] = np.transpose(data['x_train'], (0, 2, 3, 1))
data['x_validation'] = np.transpose(data['x_validation'], (0, 2, 3, 1))
data['x_test'] = np.transpose(data['x_test'], (0, 2, 3, 1))

for key, value in data.items():
    if key == 'labels':
        print(f'{key}: {len(value)}')
    else:
        print(f'{key}: {value.shape}')

In [ ]:
import numpy as np
epochs = 15
def convert_to_grid(input_images):
    total_images, image_height, image_width, channels = input_images.shape
    grid_size = int(np.ceil(np.sqrt(total_images)))
    grid_height = image_height * grid_size + (grid_size - 1)
    grid_width = image_width * grid_size + (grid_size - 1)
    image_grid = np.zeros((grid_height, grid_width, channels)) + 255
    image_index = 0
    row_start, row_end = 0, image_height
    for row in range(grid_size):
        col_start, col_end = 0, image_width
        for col in range(grid_size):
            if image_index < total_images:
                current_image = input_images[image_index]
                min_val, max_val = np.min(current_image), np.max(current_image)
                image_grid[row_start:row_end, col_start:col_end] = 255.0 * (current_image - min_val) / (max_val - min_val)
                image_index += 1
            col_start += image_width + 1
            col_end += image_width + 1
        row_start += image_height + 1
        row_end += image_height + 1
    return image_grid

examples = data['x_train'][:81, :, :, :]
print(examples.shape)
fig = plt.figure()
grid = convert_to_grid(examples)
plt.imshow(grid.astype('uint8'), cmap='gray')
plt.axis('off')
plt.gcf().set_size_inches(15, 15)
plt.show()

In [ ]:
data_seq = Sequential()
data_seq.add(MaxPool2D(pool_size=2))
data_seq.add(Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=(32, 32, 3)))
data_seq.add(Dense(500, activation='relu'))
data_seq.add(Dense(43, activation='softmax'))
data_seq.add(Flatten())
data_seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
learn_rate_stuff = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
h = data_seq.fit(data['x_train'][:10], data['y_train'][:10],batch_size=5, epochs = epochs,validation_data = (data['x_validation'], data['y_validation']),callbacks=[learn_rate_stuff], verbose=1)

plt.rcParams['figure.figsize'] = (15.0, 5.0)
plt.rcParams['image.interpolation'] = 'nearest'

fig = plt.figure()
plt.plot(h.history['accuracy'], '-o', linewidth=3.0)
plt.plot(h.history['val_accuracy'], '-o', linewidth=3.0)
plt.title('Overfitting small data', fontsize=22)
plt.legend(['train', 'validation'])
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Accuracy', fontsize=20)
plt.tick_params(labelsize=18)
plt.show()

In [ ]:
filters = [3, 5, 9, 13, 15, 19, 23, 25, 31]
data_seq = [0] * len(filters)
for i in range(len(data_seq)):
    data_seq[i] = Sequential()
    data_seq[i].add(Conv2D(32, kernel_size=filters[i], padding='same', activation='relu', input_shape=(32, 32, 3)))
    data_seq[i].add(MaxPool2D(pool_size=2))
    data_seq[i].add(Flatten())
    data_seq[i].add(Dense(500, activation='relu'))
    data_seq[i].add(Dense(43, activation='softmax'))
    data_seq[i].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
epochs = 5
h = [0] * len(data_seq)
for i in range(len(h)):
    h[i] = data_seq[i].fit(data['x_train'], data['y_train'],batch_size=5, epochs = epochs,validation_data = (data['x_validation'], data['y_validation']),callbacks=[annealer], verbose=0)
    print('Model with filters {0:d}x{0:d}, epochs={1:d}, training accuracy={2:.5f}, validation accuracy={3:.5f}'.\
      format(filters[i], epochs, max(h[i].history['accuracy']), max(h[i].history['val_accuracy'])))

In [ ]:
plt.rcParams['figure.figsize'] = (15.0, 15.0)
plt.rcParams['image.interpolation'] = 'nearest'

fig = plt.figure()
plt.subplot(2, 1, 1)
plt.plot(h[8].history['accuracy'], '-o', linewidth=3.0)
plt.plot(h[7].history['accuracy'], '-s', linewidth=3.0)
plt.plot(h[6].history['accuracy'], '-D', linewidth=3.0)
plt.plot(h[5].history['accuracy'], '-D', linewidth=3.0)
plt.plot(h[4].history['accuracy'], '-o', linewidth=3.0)
plt.plot(h[3].history['accuracy'], '-o', linewidth=3.0)
plt.plot(h[2].history['accuracy'], '-o', linewidth=3.0)
plt.plot(h[1].history['accuracy'], '-o', linewidth=3.0)
plt.plot(h[0].history['accuracy'], '-o', linewidth=3.0)
plt.legend(['filter 31', 'filter 25', 'filter 23', 'filter 19', 'filter 15', 'filter 13', 'filter 9', 'filter 5', 'filter 3'], loc='lower right', fontsize='xx-large', borderpad=2)
plt.xlabel('Epoch', fontsize=20, fontname='Times New Roman')
plt.ylabel('Training accuracy', fontsize=20, fontname='Times New Roman')
plt.yscale('linear') 
plt.ylim(0.85, 1.0)
plt.xlim(0.5, 5.3) 
plt.title('accuracy for different sizes of filters', fontsize=22)
plt.tick_params(labelsize=18)

plt.subplot(2, 1, 2)
plt.plot(h[8].history['val_accuracy'], '-o', linewidth=3.0)
plt.plot(h[7].history['val_accuracy'], '-s', linewidth=3.0)
plt.plot(h[6].history['val_accuracy'], '-D', linewidth=3.0)
plt.plot(h[5].history['val_accuracy'], '-D', linewidth=3.0)
plt.plot(h[4].history['val_accuracy'], '-o', linewidth=3.0)
plt.plot(h[3].history['val_accuracy'], '-o', linewidth=3.0)
plt.plot(h[2].history['val_accuracy'], '-o', linewidth=3.0)
plt.plot(h[1].history['val_accuracy'], '-o', linewidth=3.0)
plt.plot(h[0].history['val_accuracy'], '-o', linewidth=3.0)
plt.legend(['filter 31', 'filter 25', 'filter 23', 'filter 19', 'filter 15', 'filter 13', 'filter 9', 'filter 5', 'filter 3'], loc='lower right', fontsize='xx-large', borderpad=2)
plt.xlabel('Epoch', fontsize=20, fontname='Times New Roman')
plt.ylabel('Validation accuracyuracy', fontsize=20, fontname='Times New Roman')
plt.yscale('linear')
plt.ylim(0.75, 0.9)
plt.xlim(0.5, 5.3)
plt.tick_params(labelsize=18)
plt.show()
for i in range(len(h)):
    print('data2 filter {0:d} training accuracy = {1:.5f}'.\
          format(filters[i], np.max(h[i].history['accuracy'])))

print()
for i in range(len(h)):
    print('data2 filter {0:d} validation accuracy = {1:.5f}'.\
          format(filters[i], np.max(h[i].history['val_accuracy'])))

In [ ]:
for i in range(len(data_seq)):
    temp = data_seq[i].predict(data['x_test'])
    temp = np.argmax(temp, axis=1)
    temp = np.mean(temp == data['y_test'])
    print('data2 filter {0:d} testing accuracy = {1:.5f}'.format(filters[i], temp))

In [ ]:
for i in range(len(data_seq)):
    start = timer()
    temp = data_seq[i].predict(data['x_test'][:1, :, :, :])
    end = timer()
    print('data2 filter {0:d} classification time = {1:.5f}'.format(filters[i], end - start))

In [ ]:
for i in range(len(data_seq)):
    w = data_seq[i].get_weights()
    print(w[0].shape)
    temp = w[0].transpose(3, 0, 1, 2)
    print(temp.shape) 
    fig = plt.figure()
    grid = convert_to_grid(temp)
    plt.imshow(grid.astype('uint8'), cmap='gray')
    plt.axis('off')
    plt.gcf().set_size_inches(10, 10)
    name = 'Trained filters ' + str(filters[i]) + 'x' + str(filters[i])
    plt.title(name, fontsize=18)
    plt.show()


License plate detection


In [ ]:
plate_cascade = cv2.CascadeClassifier('Vehicle-Detection/archive/indian_license_plate.xml')

In [ ]:
def detect_plate(img, text=''):
    plate_img = img.copy()
    roi = img.copy()
    plate_rect = plate_cascade.detectMultiScale(plate_img, scaleFactor = 1.2, minNeighbors = 7)
    for (x,y,w,h) in plate_rect:
        roi_ = roi[y:y+h, x:x+w, :] 
        plate = roi[y:y+h, x:x+w, :]
        cv2.rectangle(plate_img, (x+2,y), (x+w-3, y+h-5), (51,181,155), 3)
    if text!='':plate_img = cv2.putText(plate_img, text, (x-w//2,y-h//2),cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.5, (51,181,155), 1, cv2.LINE_AA)
    return plate_img, plate

In [ ]:
def display(img_, title=''):
    img = cv2.cvtColor(img_, cv2.COLOR_BGR2RGB)
    fig = plt.figure(figsize=(10,6))
    ax = plt.subplot(111)
    ax.imshow(img)
    plt.axis('off')
    plt.title(title)
    plt.show()
img = cv2.imread('/home/lakshya/ML/Vehicle-Detection/archive/car.jpg')
display(img, 'input image')

In [ ]:
output_img, plate = detect_plate(img)
display(output_img, 'detected license plate in the input image')

In [ ]:
display(plate, 'extracted license plate from the image')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def find_contours(size_range, image):
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    min_width = size_range[0]
    max_width = size_range[1]
    min_height = size_range[2]
    max_height = size_range[3]
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:15]
    output_image = cv2.imread('contour.jpg')
    contour_x_positions = []
    valid_contours = []
    processed_characters = []
    for contour in contours:
        x, y, width, height = cv2.boundingRect(contour)
        if min_width < width < max_width and min_height < height < max_height:
            contour_x_positions.append(x)
            character_template = np.zeros((44, 24))
            character = image[y:y+height, x:x+width]
            character = cv2.resize(character, (20, 40))
            cv2.rectangle(output_image, (x, y), (width+x, y+height), (50, 21, 200), 2)
            plt.imshow(output_image, cmap='gray')
            character = cv2.subtract(255, character)
            character_template[2:42, 2:22] = character
            character_template[0:2, :] = 0
            character_template[:, 0:2] = 0
            character_template[42:44, :] = 0
            character_template[:, 22:24] = 0
            processed_characters.append(character_template)
    plt.show()
    ordered_indices = sorted(range(len(contour_x_positions)), key=lambda i: contour_x_positions[i])
    ordered_characters = [processed_characters[i] for i in ordered_indices]
    processed_characters = np.array(ordered_characters)
    return processed_characters


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def segment_characters(input_image):
    resized_image = cv2.resize(input_image, (333, 75))
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    processed_image = cv2.dilate(cv2.erode(binary_image, (3, 3)), (3, 3))
    image_width = processed_image.shape[0]
    image_height = processed_image.shape[1]
    processed_image[0:3, :] = 255
    processed_image[:, 0:3] = 255
    processed_image[72:75, :] = 255
    processed_image[:, 330:333] = 255
    contour_dimensions = [image_width / 6, image_width / 2, image_height / 10, 2 * image_height / 3]
    plt.imshow(processed_image, cmap='gray')
    plt.show()
    cv2.imwrite('contour_output.jpg', processed_image)
    characters = find_contours(contour_dimensions, processed_image)
    return characters


In [ ]:
char = segment_characters(plate)
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(char[i], cmap='gray')
    plt.axis('off')

In [ ]:
import tensorflow.keras.backend as BackEnd
train_generator_data = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1)
path = 'Vehicle-Detection/archive/data/data'
validation_generator = train_generator_data.flow_from_directory(path+'/val',  target_size=(28,28), class_mode='sparse')
train_generator = train_generator_data.flow_from_directory(path+'/train',target_size=(28,28),batch_size=1,class_mode='sparse')


In [ ]:
def f1score(y, y_pred):
    return f1_score(y, tf.math.argmax(y_pred, axis=1), average='micro') 
def custom_f1score(y, y_pred):
    return tf.py_function(f1score, (y, y_pred), tf.double)

In [ ]:
BackEnd.clear_session()
data_seq = Sequential()
data_seq.add(Conv2D(16, (22,22), input_shape=(28, 28, 3), activation='relu', padding='same'))
data_seq.add(Conv2D(32, (16,16), input_shape=(28, 28, 3), activation='relu', padding='same'))
data_seq.add(Conv2D(64, (8,8), input_shape=(28, 28, 3), activation='relu', padding='same'))
data_seq.add(Conv2D(64, (4,4), input_shape=(28, 28, 3), activation='relu', padding='same'))
data_seq.add(MaxPooling2D(pool_size=(4, 4)))
data_seq.add(Dropout(0.4))
data_seq.add(Flatten())
data_seq.add(Dense(128, activation='relu'))
data_seq.add(Dense(36, activation='softmax'))
data_seq.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(lr=0.0001), metrics=[custom_f1score])
data_seq.summary()

In [ ]:
class stop_training_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_custom_f1score') > 0.99):self.model.stop_training = True

In [ ]:
batch_size = 1
callbacks = [stop_training_callback()]
data_seq.fit_generator(train_generator,steps_per_epoch = train_generator.samples // batch_size,validation_data = validation_generator, epochs = 80, verbose=1, callbacks=callbacks)

In [ ]:

def fix_dimension(image): 
  zeroes_image = np.zeros((28,28,3))
  for i in range(3):
    zeroes_image[:,:,i] = image
  return zeroes_image
def show_results():
    dictt = {}
    char = 'QWERTYUIOPLKJHGFDSAZXCVBNM1234567890'
    for i,c in enumerate(char):
        dictt[i] = c
    output = []
    for _,chr in enumerate(char):
      lol_image = cv2.resize(chr, (28, 28), interpolation=cv2.INTER_AREA)
      image = fix_dimension(lol_image)
      image = image.reshape(1, 28, 28, 3)
      y_prob = model.predict(image)
      y_class = np.argmax(y_prob, axis=-1)
      character = dictt[y_class[0]] 
      output.append(character)
        
    detected_plate = ''.join(output)
    
    return detected_plate

print(show_results())

In [ ]:
plt.figure(figsize=(10,6))
for index,chr in enumerate(char):
    image = cv2.resize(chr, (28,28), interpolation=cv2.INTER_AREA)
    plt.subplot(3,4,index+1)
    plt.imshow(image,cmap='gray')
    plt.title(f'predicted: {show_results()[i]}')
    plt.axis('off')
plt.show()